### Part 2: Cluster Analysis

In [3]:
import sys
import csv
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [8]:
# Return a pandas dataframe containing the data set that needs to be extracted from the data_file.
# data_file will be populated with the string 'wholesale_customers.csv'.
def read_csv_2(data_file):
      
    df = pd.read_csv(data_file,index_col=False, skipinitialspace = True)
    df = df.drop(['Channel', 'Region'], axis=1)    
    df.replace(to_replace=[r' ?', r'?', r'? ', r' ', r''], value=[np.nan, np.nan, np.nan, np.nan, np.nan], regex=False, inplace=True)
    
    return df

In [30]:
# Return a pandas dataframe with summary statistics of the data.
# Namely, 'mean', 'std' (standard deviation), 'min', and 'max' for each attribute.
# These strings index the new dataframe columns. 
# Each row should correspond to an attribute in the original data and be indexed with the attribute name.
def summary_statistics(df):
    
    index_labels=['mean', 'standard deviation', 'minimum', 'maximum']
    df_summary = pd.DataFrame([df.mean().astype(int), df.std().astype(int), 
                               df.min().astype(int), df.max().astype(int) ], 
                              index=index_labels)
    
    return df_summary

### Answers

In [26]:
df = read_csv_2('Wholesale customers data.csv')
df.head(4)

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,12669,9656,7561,214,2674,1338
1,7057,9810,9568,1762,3293,1776
2,6353,8808,7684,2405,3516,7844
3,13265,1196,4221,6404,507,1788


In [31]:
#1. [10 points] Compute the mean, standard deviation, minimum, and maximum value for each
# attribute. Round the mean and standard deviation to the closest integers.
df_summary = summary_statistics(df)
df_summary.head()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
mean,12000,5796,7951,3071,2881,1524
standard deviation,12647,7380,9503,4854,4767,2820
minimum,3,55,3,25,3,3
maximum,112151,73498,92780,60869,40827,47943


In [ ]:
#2 [20 points] Divide the data points into k clusters, for k ∈ {3, 5, 10}, 
# using kmeans and agglomerative hierarchical clustering. Because the performance of kmeans (e.g. number of iterations)
# is significantly affected by the initial cluster center selection, repeat 10 executions of kmeans for
# each k value. Next, standardize each attribute value by subtracting with the mean and then
# dividing with the standard deviation for that attribute. Repeat the previous kmeans and agglomerative hierarchical clustering executions with the standardized data set. Identify which run
# resulted in the best set of clusters using the Silhouette score as your evaluation metric. Visualize
# the best set of clusters computed in the previous question. For this, construct a scatterplot for
# each pair of attributes using Pyplot. Therefore, 15 scatter plots should be constructed in total.
# Different clusters should appear with different colors in each scatter plot. Note that these plots

